In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import matplotlib.pyplot as plt


# Read the data
df = pd.read_csv('Petrol CSV w Date.csv')  # Replace with your file path

# Create datetime index from YEARS column
df['date'] = df['YEARS'].apply(
    lambda x: pd.to_datetime(f"19{x[-2:]}-01-01" if int(x[:2]) < 20 else f"20{x[-2:]}-01-01")
)



KeyError: 'YEARS'

In [ ]:
# Set the column name for consumption if needed
if 'PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit' not in df.columns:
    df = df.rename(columns={'Consumption': 'PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit'})

# Create copy of the dataframe
df_features = df.copy()

# Set date as index
df_features.set_index('date', inplace=True)

# Extract basic datetime features
df_features['year'] = df_features.index.year
df_features['quarter'] = df_features.index.quarter
df_features['month'] = df_features.index.month

# Create lag features
label = 'PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit'
df_features['consumption_lag1'] = df_features[label].shift(1)
df_features['consumption_lag2'] = df_features[label].shift(2)

# Create rolling mean features
df_features['rolling_mean_3'] = df_features[label].rolling(window=3).mean()
df_features['rolling_mean_5'] = df_features[label].rolling(window=5).mean()

# Create rolling standard deviation
df_features['rolling_std_3'] = df_features[label].rolling(window=3).std()

# Create percentage change features
df_features['pct_change'] = df_features[label].pct_change()
df_features['pct_change_rolling'] = df_features['pct_change'].rolling(window=3).mean()

# Create year-based features
df_features['years_since_start'] = (df_features.index.year - df_features.index.year.min())

# Select features for the model
feature_columns = ['year', 'quarter', 'month', 
                  'consumption_lag1', 'consumption_lag2',
                  'rolling_mean_3', 'rolling_mean_5', 'rolling_std_3',
                  'pct_change', 'pct_change_rolling', 'years_since_start']

# Split into features (X) and target (y)
X = df_features[feature_columns]
y = df_features[label]

# Remove any rows with NaN values
X = X.dropna()
y = y[X.index]

# Split into train and test sets (using last 5 years as test)
train_size = len(X) - 5
X_train = X[:train_size]
X_test = X[train_size:]
y_train = y[:train_size]
y_test = y[train_size:]

# Scale the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
dtest = xgb.DMatrix(X_test_scaled, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.01,
    'max_depth': 4,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eta': 0.1
}

# Train the model
model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=50,
    verbose_eval=100
)

# Make predictions
train_predictions = model.predict(dtrain)
test_predictions = model.predict(dtest)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
train_mae = mean_absolute_error(y_train, train_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)
train_r2 = r2_score(y_train, train_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print metrics
print("\nModel Performance Metrics:")
print(f"Training RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")
print(f"Training MAE: {train_mae:.2f}")
print(f"Test MAE: {test_mae:.2f}")
print(f"Training R2 Score: {train_r2:.2f}")
print(f"Test R2 Score: {test_r2:.2f}")

# Plot feature importance
importance = model.get_score(importance_type='gain')
importance = pd.DataFrame(list(importance.items()), columns=['Feature', 'Importance'])
importance = importance.sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance)
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

# Plot actual vs predicted values
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test.values, label='Actual', marker='o')
plt.plot(y_test.index, test_predictions, label='Predicted', marker='o')
plt.title('Actual vs Predicted Values')
plt.xlabel('Date')
plt.ylabel('Consumption')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Make future predictions (next 10 years)
last_date = X_test.index[-1]
future_dates = pd.date_range(start=last_date + pd.DateOffset(years=1), periods=10, freq='Y')

# Prepare future features
future_features = pd.DataFrame(index=future_dates)
future_features['year'] = future_features.index.year
future_features['quarter'] = future_features.index.quarter
future_features['month'] = future_features.index.month
future_features['years_since_start'] = (future_features.index.year - df_features.index.year.min())

# Initialize with last known values
last_known_consumption = y_test.iloc[-1]
future_features['consumption_lag1'] = last_known_consumption
future_features['consumption_lag2'] = y_test.iloc[-2]
future_features['rolling_mean_3'] = y_test.iloc[-3:].mean()
future_features['rolling_mean_5'] = y_test.iloc[-5:].mean()
future_features['rolling_std_3'] = y_test.iloc[-3:].std()
future_features['pct_change'] = (last_known_consumption - y_test.iloc[-2]) / y_test.iloc[-2]
future_features['pct_change_rolling'] = future_features['pct_change']

# Scale future features
future_features_scaled = scaler.transform(future_features[feature_columns])
future_dmatrix = xgb.DMatrix(future_features_scaled)

# Make predictions
future_predictions = model.predict(future_dmatrix)

# Plot historical and future predictions
plt.figure(figsize=(15, 7))
plt.plot(y.index, y.values, label='Historical Data', color='blue')
plt.plot(y_test.index, test_predictions, label='Test Predictions', color='green')
plt.plot(future_dates, future_predictions, label='Future Predictions', color='red', linestyle='--')
plt.title('Historical Data and Future Predictions')
plt.xlabel('Date')
plt.ylabel('Consumption')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Print future predictions
future_results = pd.DataFrame({
    'Date': future_dates,
    'Predicted_Consumption': future_predictions
})
print("\nFuture Predictions:")
print(future_results)